In [ ]:
# Provide the bronze and silver paths in your Synapse linked Azure Data lake
bronze_path = 'abfss://xxxxxx@yyyyyyyy.dfs.core.windows.net/bronze/'
silver_path = 'abfss://xxxxxx@yyyyyyyy.dfs.core.windows.net/silver/'

In [ ]:

# Create a table called "ny_taxis_2019" and load the data from the bronze container 
from pyspark.sql.types import *

table_name = 'ny_taxis_2019'

# TODO read the data from the bronze container


In [1]:
#CLEAN-UP and ENRICH DATA

# TODO Discard some data that wont be used, during our anlysis: remove following columns: doLocationId, puLocationId, storeAndFwdFlag and vendorID


# TODO Measure the Tip percentage, based on the columns totalAmount and tipAmount. Create a column called "tipPercentage"

# TODO Measure the price per mile for every service offered, based on the colums totalAmount and tripDistance. Create a column called "pricePerMile"

In [ ]:
# TODO Convert the Data to delta format and save it to the silver container

#Expected Result
spark.sql("SELECT totalAmount, tripDistance, pricePerMile, tipPercentage  FROM silver_" + table_name).show(5)